In [1]:
from collections import namedtuple, Counter, defaultdict
import math
import numpy as np
from sklearn import tree
import random

In [51]:
from sklearn.model_selection import train_test_split
import pandas as pd

d_meta = pd.read_csv('MetaClassifier_Dataset_NEWSize3.csv')
labels = list(d_meta)
data = d_meta.as_matrix()[:,:] 

Y = data[:, -1]
X = data[:,0:-1]
#
#training_set_X = X
#training_set_Y = Y
training_set_X, test_set_X, training_set_Y, test_set_Y, = train_test_split(X, Y, test_size=0.33)

In [52]:
#This block calculates the costs for the different models as well as the conditional cost 

feature_costs = {'1':20, '2':20,'3':0,'4':30, '5':45, '6':45,'7':10,'8':45,'9':10,'10':12,'11':10,'12':10}
model_info = {}
with open("Models_MetaClassifier_NEW.csv", "r") as ins:
    for line in ins:
        info = line.rstrip('\n').rstrip('\r').split(",")
        model_info[info[0]] = info[1:]

global model_cost 
model_cost = []
for i in labels[:-1]:
    model_num = i[i.index('l')+1:]
    model_string = 'Model '+str(model_num)
    cost = 0
    for feature in model_info[model_string]:
        if feature != "":
            cost += feature_costs[feature]
    model_cost.append(cost)
    
global model_condition_cost 
model_condition_cost = {}
for i in labels[:-1]:
    model_num1 = i[i.index('l')+1:]
    model_string = 'Model '+str(model_num1)
    model_condition_cost[int(model_num1)] = {}
    for j in labels[:-1]:
        model_num2 = j[j.index('l')+1:]
        model_string2 = 'Model '+str(model_num2)
        
        features1 = model_info[model_string]
        features2 = model_info[model_string2]
        additional_cost = 0
        for f in features2:
            if f not in features1:
                additional_cost += feature_costs[f]
                
        model_condition_cost[int(model_num1)][int(model_num2)] = additional_cost


In [59]:
class DecisionNode(object):
    """Makes Decision Node Class"""
    def __init__(self,feature=None,left=None,right=None,classes = None):
        self.left = left
        self.right = right
        self.feature = feature
        self.classes = classes
    
    def is_leaf(self):
        return self.left == None and self.right == None

    def __repr__(self):
        if self.feature == None:
            return str(Counter(self.classes))
        return "Decision node for feature " + str(self.feature)

class DecisionTree(object):
    """Decision Tree Class"""
    def __init__(self):
        self.root_node = None
        
    def predict(self, X):
        predicted_classes = []

        for sample in X:
            c = None
            current_node = self.root_node
            while c is None:
                if current_node.is_leaf():
                    try:
                        c = int(current_node.classes)
                    except:
                        ones = sum(current_node.classes)
                        zeros = len(current_node.classes)-ones
                        if ones > zeros:
                            c = 1
                        else:
                            c = 0
                    
                else:
                    key_value = sample[current_node.feature]
                    if key_value == 0:
                        current_node = current_node.left
                    elif key_value == 1:
                        current_node = current_node.right
            predicted_classes.append(c)
        return predicted_classes
    
    def fit(self,samples,outcome_variable, depth_max):
        """Takes in training data and builds a decision tree
        samples = X values as list of list
        outcome_varaible = Y value
        """
        training_samples = [(s, t) for s, t in zip(samples, outcome_variable)]
        predicting_features = list(range(len(samples[0])))
        random.shuffle(predicting_features)
        self.root_node = self.build_decision_tree(training_samples,predicting_features,0, depth_max)
        
    def build_decision_tree(self,samples,features, depth, depth_max):
        classes = [sample[1] for sample in samples]
        
        #if we only have one class, return just a decision node 
        if len(set(classes)) == 1:
            root_node = DecisionNode(feature=None, left=None, right=None, classes = classes)
        
        #if the depth is greater than the max_depth, return a decision node
        elif depth >= depth_max:
            return  DecisionNode(feature=None, left=None, right=None, classes = [sample[1] for sample in samples])
        
        #if there are no more remaining features to try, return decision node
        elif features == []:
            return  DecisionNode(feature=None, left=None, right=None, classes = [sample[1] for sample in samples])
        
        #otherwise, recurse
        else:
            best_feature = self.select_best_feature(samples,features,classes) #determine best feature
            print "BEST FEATURE THIS ITERATION IS ", best_feature
            
            #if the feature has a value of info_gain/estimated_cost that is <= 0, stop the algorithm
            if best_feature == 'stop':
                root_node = DecisionNode(feature=None, left=None, right=None, classes = [sample[1] for sample in samples])
                return root_node
            
            best_feature_values = [s[0][best_feature] for s in samples]
            
            #if we are at pure leaf
            if len(best_feature_values) == 1:
                root_node = DecisionNode(feature = best_feature, classes = best_feature_values[0])
           
            else:
                #do left hand side
                left_samples = [s for s in samples if s[0][best_feature] == 0]
                left_node = self.build_decision_tree(left_samples,features, depth + 1,depth_max)
                
                #do right hand side
                right_samples = [s for s in samples if s[0][best_feature] == 1]
                right_node = self.build_decision_tree(right_samples,features, depth + 1,depth_max)

                root_node = DecisionNode(feature = best_feature, classes = best_feature_values, left = left_node, right= right_node)

        return root_node
    
    
    def print_tree(self,labels):
        curr_node = self.root_node
        print self.__str__(curr_node,0)

    def __str__(self, node, depth=0):
        ret = ""
        # Print right branch
        if node.right != None:
            ret += self.__str__(node.right,depth + 1)
        # Print own value
        if node.feature != None:
            ret += "\n" + ("    "*depth) + str(node.feature)
        else:
            ret += "\n" + ("    "*depth) + "Class: " + str(node)
        # Print left branch
        if node.left != None:
            ret += self.__str__(node.left,depth + 1)
        return ret
    
    
    def select_best_feature(self, samples, features, classes):
        """
        Find score for all remaining features, choose the one that maximizes
        the score function and delete this feature from consideration
        """
        gain_factors = [(self.score_function(samples, feat, classes, features), feat)
                        for feat in features]
        gain_factors.sort()
        #print "GAIN FACTORS", gain_factors
        
        best_feature = gain_factors[-1][1]
        
        if gain_factors[-1][0] <= 0:
            return "stop"
        #features.pop(features.index(best_feature))
        return best_feature


    def information_gain(self, samples, feature, classes):
        """
        Information gain is the measure of the difference in entropy from before
        to after the samples are split on the given feature values. In other
        words, how much uncertainty in the samples was reduced after splitting
        them on the given feature.
        """
        N = len(samples)
        samples_partition = defaultdict(list)
        for s in samples:
            samples_partition[s[0][feature]].append(s)

        feature_entropy = 0.0
        subclasses = []
        #print "NUMB PART",len(samples_partition)
        for partition in samples_partition.values():
            
            sub_classes = [s[1] for s in partition]
            sub_entropy = self.entropy(sub_classes)
            subclasses.append(Counter(sub_classes))
            #print "SUBENTROPY",sub_entropy
            feature_entropy += (len(partition) / float(N)) * sub_entropy 
            #print feature_entropy
        
        
        p = self.entropy(classes)

        #if round(p,16) < round(feature_entropy,16):
        #    print "PARENT HAS SMALLER ENTROPY THAN CHILD", p, feature_entropy, Counter(classes), subclasses
            
        return p, feature_entropy

    @staticmethod
    def entropy(dataset):
        """Measure of the amount of uncertainty in the given dataset."""

        N = len(dataset)
        counter = Counter(dataset)
        return sum([-1.0*(counter[k] / float(N))*math.log(counter[k] / float(N),2) for k in counter])
    #
    def cost(self, feature):
        global model_cost
        return model_cost[feature]
    
    def cost_conditional(self, feature, feature2):
        global model_condition_cost
        return model_condition_cost[feature][feature2]
    
    def P_L(self, samples, feature, classes):
        parent_entrop, child_entrop = self.information_gain(samples, feature, classes)
        pl = 1- ((child_entrop)/float(parent_entrop))
        return pl
    
    
    def P_L_conditional(self, samples, feature, classes, features_left):
        samples_partition = defaultdict(list)
        for s in samples:
            samples_partition[s[0][feature]].append(s)
         
        sums = 0
        #print "ORIG"
        for i in samples_partition: #either 0 or 1
            #get subclasses
            sub_classes = [s[1] for s in samples_partition[i]]
            
            #for each feature left
            for f in features_left:
                
                if len(list(set(sub_classes))) > 1:
                    pl = self.P_L(samples_partition[i], f, sub_classes)
                
                #if its a pure class, pl = 1
                else:
                    pl = 1
                sums += pl*self.cost_conditional(feature,f)
        return sums

    
    
    def Estimated_Cost(self, samples, feature, classes, features_left):
        PL = self.P_L(samples, feature, classes)
        #if PL > 1 or PL < 0:
        #    print "PL", PL
        if PL == 1:
            return PL*self.cost(feature)
        
        first_part = PL*self.cost(feature)
        second_part = (1-PL)*self.P_L_conditional(samples, feature, classes, features_left) 
        
        return first_part + second_part
    
    
    
    def score_function(self,samples, feature, classes, features_left):
        #print "CURRENT FEATURE IS", feature
        info_gain1, info_gain2 = self.information_gain(samples, feature, classes)
        #print 'INFORMATION GAIN PARENT',info_gain1,"CHILD",info_gain2
        estimated_cost = self.Estimated_Cost(samples, feature, classes, features_left)
        if estimated_cost == 0:
            return 0
        if estimated_cost < 0:
            print "estimated cost < 0"
        #print "GAIN AND COST",info_gain1-info_gain2, estimated_cost
        return (info_gain1-info_gain2)/float(estimated_cost)
    
    

In [15]:
"""Function to print the Decision Tree Classifier from sklearn to compare results """
def get_code(tree, feature_names):
        left      = tree.tree_.children_left
        right     = tree.tree_.children_right
        threshold = tree.tree_.threshold
        features  = [feature_names[i] for i in tree.tree_.feature]
        value = tree.tree_.value

        def recurse(left, right, threshold, features, node):
                if (threshold[node] != -2):
                        print "if ( " + features[node] + " <= " + str(threshold[node]) + " ) {"
                        if left[node] != -1:
                                recurse (left, right, threshold, features,left[node])
                        print "} else {"
                        if right[node] != -1:
                                recurse (left, right, threshold, features,right[node])
                        print "}"
                else:
                        print "return " + str(value[node])

        recurse(left, right, threshold, features, 0)
        

In [ ]:
###Small Test Examples to Check Work

samples = np.array([[0,0,1,0,1],[1,1,1,1,0],[1,0,0,1,0],[1,0,0,1,0],[1,0,0,1,0]])
targets = np.array([1,0,1,0,1])

#samples = [[0,0,0,0,1],[1,1,1,1,0],[0,0,0,1,0],[1,0,0,1,0]]
#targets = [1,0,0,1]

#samples = [[0,0,1,0,1],[1,1,1,1,0],[0,0,0,1,0],[1,0,0,1,0]]
#targets = [1,0,1,1]

#samples = [[0,0,1,0,1],[1,1,1,1,0],[0,0,0,1,1],[1,0,0,1,0],[0,1,1,1,0],[1,1,0,0,0]]
#targets = [1,0,1,1,0,1]

#samples = [[1,1,1,0,1],[1,1,0,1,0],[0,0,0,1,1],[0,0,0,1,0],[0,1,1,1,0],[1,1,0,0,0]]
#targets = [1,0,1,0,0,1]


d = DecisionTree()
d.fit(samples,targets,[],3)
d.print_tree(range(4))

print "\nSklearn tree"
dtree = tree.DecisionTreeClassifier(criterion = 'entropy')
dtree = dtree.fit(samples,targets)
get_code(dtree,['0','1','2','3','4','5'])

print d.predict([[0,0,0,1,0]])

In [63]:
#How to Fit and Test Model

d = DecisionTree()
d.fit(training_set_X,training_set_Y,1)
d.print_tree(labels)

y_pred = d.predict(test_set_X)

count = []
recall = []
for i in range(len(y_pred)):
    if y_pred[i] == test_set_Y[i]:
        count.append(1)
    else:
        count.append(0)
        
    if test_set_Y[i] == 1:
        if y_pred[i] == 1:
            recall.append(1)
        else:
            recall.append(0)
            
print sum(count)/float(len(count))
print sum(recall)/float(len(recall))

BEST FEATURE THIS ITERATION IS  224

    Class: Counter({1: 130, 0: 17})
224
    Class: Counter({0: 226, 1: 11})
0.910526315789
0.898734177215
